In [ ]:
#@title PyG 실습에 필요한 패키지 설치
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
print(TORCH)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
print(CUDA)

!pip install torch-scatter     -q -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -q -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -q -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -q -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

1.9.0
cu102


In [ ]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric
import torch_geometric.transforms as T

from tqdm import tqdm_notebook as tqdm

from torch import Tensor
from torch.nn import Parameter as Param
from torch_geometric.datasets import Planetoid, TUDataset
from torch_geometric.data import DataLoader
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.conv import MessagePassing

torch.manual_seed(42)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
dataset = 'Cora'

path = os.path.join('.','data', dataset)
dataset = Planetoid(path, dataset)
data = dataset[0]
data = data.to(device)

### GNNM

$$
x_v^{t+1}=f_w(l_v,l_{co(v)},x_{ne(v)}^t,l_{ne(v)}) \\
o_v^t = g_w(x_v^t,l_v)
$$



In [ ]:
class MLP(nn.Module):
    '''
    dims = [input, hid, hid, hid, out]
    0 Linear(input, hid)
    0 Tanh
    1 Linear(hid, hid)
    1 Tanh
    2 Linear(hid, hid)
    2 Tanh
    3 Linear(hid, out)
    => propagation : node state (node featrue)
    => output : node score (이후 softmax 적용)
    '''
    def __init__(self, input_dim, hid_dims, out_dim):
        super(MLP, self).__init__()

        self.mlp = nn.Sequential()
        dims = [input_dim] + hid_dims + [out_dim]
        for i in range(len(dims)-1):
            self.mlp.add_module('lay_{}'.format(i), nn.Linear(in_features=dims[i], out_features=dims[i+1]))
            if i+2 < len(dims):
                self.mlp.add_module('drop_{}'.format(i), nn.Dropout())
                self.mlp.add_module('act_{}'.format(i), nn.Tanh())
            
    def reset_parameters(self):
        for i, l in enumerate(self.mlp):
            if type(l) == nn.Linear:
                nn.init.xavier_normal_(l.weight)
    
    def forward(self, x):
        return self.mlp(x)

In [ ]:
class GNNM(MessagePassing):
    def __init__(self, n_nodes, out_channels, features_dim, hid_dims,
                 max_iter=50, eps=1e-3, aggr='add', bias=True, **kwargs):
        super(GNNM, self).__init__(aggr=aggr, **kwargs)

        self.node_states = Param(torch.zeros((n_nodes, features_dim)), requires_grad=False)
        self.out_channels = out_channels
        self.eps = eps # contraction map
        self.max_iter = max_iter

        self.transition = MLP(features_dim, hid_dims, features_dim)
        self.readout = MLP(features_dim, hid_dims, out_channels)

        self.reset_paramters()
        print(self.transition)
        print(self.readout)

    def reset_paramters(self):
        self.transition.reset_parameters()
        self.readout.reset_parameters()
    
    def forward(self):
        edge_index = data.edge_index
        edge_weight = data.edge_attr
        node_states = self.node_states # 0으로 초기화된 x(0) : node feature matrix

        # Forward - repeat(max_iter는 최대 iteration)
        for i in range(self.max_iter):
            m = self.propagate(edge_index, x=node_states, edge_weight=edge_weight,
                               size=None)
            new_states = self.transition(m)
            
            with torch.no_grad():
                distance = torch.norm(new_states-node_states, dim=1)
                convergence = distance < self.eps
            node_states = new_states
            if convergence.all():
                break
        
        out = self.readout(node_states)

        return F.log_softmax(out, dim=-1)
    
    def message(self, x_j, edge_weight):
        return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j
    
    def message_and_aggregate(self, adj_t, x):
        # 이 메소드 사용 안됨
        return matmul(adj_t, x, reduce=self.aggr)
    
    def __repr__(self):
        return '{}({}, num_layers={})'.format(self.__class__.name__,
                                              self.out_channels,
                                              self.num_layers)

In [ ]:
lr = 0.001
EPOCHS = 1000

num_nodes = data.num_nodes
num_classes = dataset.num_classes
features_dim = 32
hid_dims = [64,64,64,64,64]
max_iter = 100
eps=0.01

print(f'# of nodes : {num_nodes}')
print(f'# of classes : {num_classes}')

# of nodes : 2708
# of classes : 7


In [ ]:
model = GNNM(n_nodes=num_nodes, out_channels=num_classes,
             features_dim=features_dim,
             hid_dims=hid_dims, eps=eps).to(device)
opt = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

MLP(
  (mlp): Sequential(
    (lay_0): Linear(in_features=32, out_features=64, bias=True)
    (drop_0): Dropout(p=0.5, inplace=False)
    (act_0): Tanh()
    (lay_1): Linear(in_features=64, out_features=64, bias=True)
    (drop_1): Dropout(p=0.5, inplace=False)
    (act_1): Tanh()
    (lay_2): Linear(in_features=64, out_features=64, bias=True)
    (drop_2): Dropout(p=0.5, inplace=False)
    (act_2): Tanh()
    (lay_3): Linear(in_features=64, out_features=64, bias=True)
    (drop_3): Dropout(p=0.5, inplace=False)
    (act_3): Tanh()
    (lay_4): Linear(in_features=64, out_features=64, bias=True)
    (drop_4): Dropout(p=0.5, inplace=False)
    (act_4): Tanh()
    (lay_5): Linear(in_features=64, out_features=32, bias=True)
  )
)
MLP(
  (mlp): Sequential(
    (lay_0): Linear(in_features=32, out_features=64, bias=True)
    (drop_0): Dropout(p=0.5, inplace=False)
    (act_0): Tanh()
    (lay_1): Linear(in_features=64, out_features=64, bias=True)
    (drop_1): Dropout(p=0.5, inplace=False)
  

In [ ]:
def train():
    model.train()
    opt.zero_grad()
    loss_fn(model()[data.train_mask], data.y[data.train_mask]).backward()
    opt.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [ ]:
for epoch in tqdm(range(EPOCHS)):
    train()

    if (epoch % 100) == 0:
        accs = test()
        train_acc = accs[0]
        val_acc = accs[1]
        test_acc = accs[2]
        print(f"Epoch: {epoch+100}, Train Acc : {train_acc:.3f}, Val Acc : {val_acc:.3f}, Test Acc : {test_acc:.3f}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 100, Train Acc : 0.114, Val Acc : 0.144, Test Acc : 0.148
Epoch: 200, Train Acc : 0.150, Val Acc : 0.116, Test Acc : 0.121
Epoch: 300, Train Acc : 0.186, Val Acc : 0.118, Test Acc : 0.131
Epoch: 400, Train Acc : 0.143, Val Acc : 0.092, Test Acc : 0.100
Epoch: 500, Train Acc : 0.157, Val Acc : 0.096, Test Acc : 0.100
Epoch: 600, Train Acc : 0.136, Val Acc : 0.088, Test Acc : 0.093
Epoch: 700, Train Acc : 0.150, Val Acc : 0.152, Test Acc : 0.145
Epoch: 800, Train Acc : 0.143, Val Acc : 0.310, Test Acc : 0.317
Epoch: 900, Train Acc : 0.129, Val Acc : 0.232, Test Acc : 0.232
Epoch: 1000, Train Acc : 0.129, Val Acc : 0.124, Test Acc : 0.123


### Gated Graph Neural Network

\begin{align}\begin{aligned}\mathbf{h}_i^{(0)} &= \mathbf{x}_i \, \Vert \, \mathbf{0}\\\mathbf{m}_i^{(l+1)} &= \sum_{j \in \mathcal{N}(i)} e_{j,i} \cdot
\mathbf{\Theta} \cdot \mathbf{h}_j^{(l)}\\\mathbf{h}_i^{(l+1)} &= \textrm{GRU} (\mathbf{m}_i^{(l+1)},
\mathbf{h}_i^{(l)})\end{aligned}\end{align}

In [ ]:
class GatedGraphConv(MessagePassing):
    def __init__(self, out_channels, num_layers, aggr='add',
                 bias=True, **kwargs):
        super(GatedGraphConv, self).__init__(aggr=aggr, **kwargs)

        self.out_channels = out_channels
        self.num_layers = num_layers

        self.weight = Param(Tensor(num_layers, out_channels, out_channels))
        self.rnn = torch.nn.GRUCell(input_size=out_channels,
                                    hidden_size=out_channels,
                                    bias=bias)
        self.reset_parameters()
    
    def reset_parameters(self):
        
        # bound = 1.0 / math.sqrt(out_channels)
        # tensor.data.uniform_(-bound, bound)
        
        uniform(self.out_channels, self.weight)
        self.rnn.reset_parameters()
    
    def forward(self, data):
        #-(x -init-> h)------------------------------------------------------#
        x = data.x
        edge_index = data.edge_index
        edge_weight = data.edge_attr

        if x.size(-1) > self.out_channels:
            raise ValueError('The number of input channels is not allowed to\
            \n be larger than the number of output channels')
        
        if x.size(-1) < self.out_channels:
            zero = x.new_zeros(x.size(0), self.out_channels - x.size(-1))
            x = torch.cat([x, zero], dim=1)
        #-------------------------------------------------------------------#
        for i in range(self.num_layers):
            
            m = torch.matmul(x, self.weight[i])

            # propagate and GRU
            # (h -> a)
            m = self.propagate(edge_index, x=m, edge_weight=edge_weight,
                               size=None)
            x = self.rnn(m, x)
        
        return x
    
    def message(self, x_j, edge_weight):
        return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j
    
    def message_and_aggregate(self, adj_t, x):
        return matmul(adj_t, x, reduce=self.aggr)
    
    def __repr__(self):
        return '{}({}, num_layers={})'.format(self.__class__.__name__,
                                              self.out_channels,
                                              self.num_layers)
        
class GGNN(torch.nn.Module):
    def __init__(self, input_dim, hid_dims, out_dim, num_layers):
        super(GGNN, self).__init__()
        self.conv = GatedGraphConv(input_dim, num_layers)
        self.mlp = MLP(input_dim, hid_dims, out_dim)
    
    def forward(self):
        x = self.conv(data)
        x = self.mlp(x)
        return F.log_softmax(x, dim=-1)

In [ ]:
lr=0.0001

In [ ]:
model = GGNN(input_dim=data.num_features,
             hid_dims=[32,32,32],
             out_dim=dataset.num_classes,
             num_layers=3).to(device)

opt = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=30, gamma=0.5)
loss_fn = nn.CrossEntropyLoss()

def train():
    model.train()
    opt.zero_grad()
    loss_fn(model()[data.train_mask], data.y[data.train_mask]).backward()
    opt.step()
    scheduler.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


for epoch in tqdm(range(500)):
    train()
    
    if (epoch % 10) == 0:
        accs = test()
        train_acc = accs[0]
        val_acc = accs[1]
        test_acc = accs[2]
        print('Epoch: {:03d}, Train Acc: {:.5f}, '
                'Val Acc: {:.5f}, Test Acc: {:.5f}'.format(epoch, train_acc,
                                                            val_acc, test_acc))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 000, Train Acc: 0.17857, Val Acc: 0.17200, Test Acc: 0.15600
Epoch: 010, Train Acc: 0.65000, Val Acc: 0.56200, Test Acc: 0.55400
Epoch: 020, Train Acc: 0.77857, Val Acc: 0.70600, Test Acc: 0.67500
Epoch: 030, Train Acc: 0.86429, Val Acc: 0.73400, Test Acc: 0.69700
Epoch: 040, Train Acc: 0.90000, Val Acc: 0.74600, Test Acc: 0.70600
Epoch: 050, Train Acc: 0.93571, Val Acc: 0.74600, Test Acc: 0.70800
Epoch: 060, Train Acc: 0.95714, Val Acc: 0.75600, Test Acc: 0.71400
Epoch: 070, Train Acc: 0.95714, Val Acc: 0.76000, Test Acc: 0.72000
Epoch: 080, Train Acc: 0.95714, Val Acc: 0.75400, Test Acc: 0.71400
Epoch: 090, Train Acc: 0.96429, Val Acc: 0.75000, Test Acc: 0.71400
Epoch: 100, Train Acc: 0.97143, Val Acc: 0.74800, Test Acc: 0.71300
Epoch: 110, Train Acc: 0.97143, Val Acc: 0.75000, Test Acc: 0.71800
Epoch: 120, Train Acc: 0.97143, Val Acc: 0.75400, Test Acc: 0.72000
Epoch: 130, Train Acc: 0.97143, Val Acc: 0.74800, Test Acc: 0.72200
Epoch: 140, Train Acc: 0.97857, Val Acc: 0.75200